In [8]:
from Bio.KEGG import Enzyme
from Bio.KEGG import REST
import pandas as pd
from cobrascape.ensemble import save_json_obj, load_json_obj

In [2]:
save_file=False

In [3]:
### Get all pathways and their names
mtb_pathways_list = REST.kegg_list("pathway", "mtu").read()
mtb_pathways = []
mtb_path_to_name = {}
for line in mtb_pathways_list.rstrip().split("\n")[:]:
    entry, description = line.split("\t")
    mtb_pathways.append(entry)
    mtb_path_to_name.update({entry: description.split(" - Mycobacterium tuberculosis H37Rv")[0]})

In [4]:
### Get all pathways and their genes
mtb_pathways_link = REST.kegg_link("mtu", "pathway").read()
kegg_path_to_genes = {}
for line in mtb_pathways_link.rstrip().split("\n")[:]:
    entry, description = line.split("\t")
    path_name = mtb_path_to_name[entry]
    if path_name not in kegg_path_to_genes.keys():
        kegg_path_to_genes[path_name] = [description.split(":")[1]]
    else:
        kegg_path_to_genes[path_name].append(description.split(":")[1])
    # print(line)

In [5]:
if save_file==True:
    save_json_obj(kegg_path_to_genes, "cobra_model/kegg_path_to_genes.json")

In [ ]:
### Get all pathways and their genes
mtb_genes_path_link = REST.kegg_link("pathway", "mtu").read()
kegg_gene_to_paths = {}
for line in mtb_genes_path_link.rstrip().split("\n")[:]:
    entry, description = line.split("\t")
    gene_name = entry.split(":")[1]
    path_name = mtb_path_to_name[description]
    if gene_name not in kegg_gene_to_paths.keys():
        kegg_gene_to_paths[gene_name] = [path_name]
    else:
        kegg_gene_to_paths[gene_name].append(path_name)
    # print(line)

In [6]:
if save_file==True:
    save_json_obj(kegg_gene_to_paths, "cobra_model/kegg_gene_to_paths.json")

Load and combine

In [38]:
kegg_path_to_genes = load_json_obj("cobra_model/kegg_path_to_genes.json")
kegg_gene_to_paths = load_json_obj("cobra_model/kegg_gene_to_paths.json")

biocyc_pathways_df = pd.read_csv("cobra_model/TB_genes_2_pathways_biocyc.txt", sep="\t")
biocyc_pathways_df.set_index("Accession-1",inplace=True)
biocyc_pathways_df.index.name=None
gene_to_pathways = {}
for gene, gene_row in biocyc_pathways_df.iterrows():
    if type(gene_row["Pathways of a reaction"]) is not float:
        superpathway_list = gene_row["Pathways of gene"].split(" // ")
        superpathway_list = [x.strip('"') for x in superpathway_list]
        pathway_list = gene_row["Pathways of a reaction"].split(" // ")
        pathway_list = [x.strip('"') for x in pathway_list]
        pathway_list.extend(superpathway_list)
        if gene in kegg_gene_to_paths.keys():
            pathway_list.extend(kegg_gene_to_paths[gene])
            pathway_list = list(set(pathway_list))
        gene_to_pathways.update({gene: pathway_list})
    else:
        pathway_list = []
        if gene in kegg_gene_to_paths.keys():
            pathway_list.extend(kegg_gene_to_paths[gene])
            pathway_list = list(set(pathway_list))
        gene_to_pathways.update({gene: pathway_list})
        
if save_file==True:
    save_json_obj(gene_to_pathways, "cobra_model/gene_to_pathways.json")

In [39]:
save_json_obj(gene_to_pathways, "cobra_model/gene_to_pathways.json")

In [40]:
gene_to_pathways["Rv1908c"]

['',
 'Phenylalanine metabolism',
 'Drug metabolism - other enzymes',
 'Tryptophan metabolism',
 'superoxide radicals degradation',
 'Metabolic pathways',
 'reactive oxygen species degradation',
 'Biosynthesis of secondary metabolites',
 'isoniazid activation']

In [42]:
# filter out the pathways that only have 1 gene
pathway_to_genes = {}
for gene, pathway_list in gene_to_pathways.items():
    for pathway in pathway_list:
        if pathway not in pathway_to_genes.keys():
            pathway_to_genes[pathway] = []
        else:
            pathway_to_genes[pathway].append(gene)
            
drop_pathways = []
for pathway, gene_list in pathway_to_genes.items():
    if len(list(set(gene_list)))<2:
        drop_pathways.append(pathway)
        
gene_to_pathways_filt = gene_to_pathways
for gene, pathway_list in gene_to_pathways.items():
    for pathway in pathway_list:
        if pathway in drop_pathways:
            gene_to_pathways_filt[gene].remove(pathway)

In [43]:
if save_file==True:
    save_json_obj(gene_to_pathways_filt, "cobra_model/gene_to_pathways_filt.json")

In [45]:
gene_to_pathways_filt["Rv1908c"]

['',
 'Phenylalanine metabolism',
 'Drug metabolism - other enzymes',
 'Tryptophan metabolism',
 'superoxide radicals degradation',
 'Metabolic pathways',
 'reactive oxygen species degradation',
 'Biosynthesis of secondary metabolites']

In [23]:
pathway_to_genes["isoniazid activation"]

['Rv1908c']

In [32]:
pathway_to_genes["superoxide radicals degradation"]

['Rv3846', 'Rv2633c', 'Rv2633c', 'Rv0432', 'Rv0432', 'Rv1908c', 'Rv1908c']

In [31]:
len(drop_pathways)

70

In [16]:
pathway_list.count(pathway)

NameError: name 'pathway' is not defined